'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


In [2]:
pulp.pulpTestAll()

.

Solver <class 'pulp.apis.coin_api.PULP_CBC_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_DLL'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_PY'> unavailable
Solver <class 'pulp.apis.coin_api.COIN_CMD'> unavailable
Solver <class 'pulp.apis.coin_api.COINMP_DLL'> unavailable
Solver <class 'pulp.apis.glpk_api.GLPK_CMD'> available
Solver <class 'pulp.apis.xpress_api.XPRESS'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI_CMD'> unavailable
Solver <class 'pulp.apis.glpk_api.PYGLPK'> unavailable
Solver <class 'pulp.apis.coin_api.YAPOSIB'> unavailable
Solver <class 'pulp.apis.choco_api.PULP_CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.choco_api.CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.mipcl_api.MIPCL_CMD'> unavailable
Solver <class 'pulp.apis.mosek_api.MOSEK'> unavailable
Solver <class 'pulp.apis.scip_api.SCIP_CMD'> unavailable
	 Testing in

.....

	 Testing MIP solution
	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict


........

...

......

	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)


................................

{'a': 53.0, 'b': 45.3, 'c': 459.2}


.............
----------------------------------------------------------------------
Ran 86 tests in 2.628s

OK


# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [77]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10


In [27]:
from pulp import *

In [99]:
prob = LpProblem("The Whiskas Problem",LpMinimize)
LpVariable("example", None, 100)
x1=LpVariable("ChickenPercent",0,None)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("SardinesPercent",0)
prob += 0.016*x1 + 0.008*x2+0.02*x3, "Total Cost of Ingredients per can"
prob += x1 + x2 +x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.15*x3  >= 8.0, "ProteinRequirement"
prob += 0.10*x1 + 0.01*x2 + 0.80*x3  >= 6.0, "FatRequirement" 
prob += 0.001*x1 + 0.005*x2 + 0.002*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.004*x3 <= 0.4, "SaltRequirement"
prob += 0.03*x1  + 0.26*x2  + 0.5*x3  >= 0.8 , "Vitamin B Requirement"


prob.writeLP("WhiskasModel.lp")
prob.solve()

print("Status:", LpStatus[prob.status])
for v in prob.variables():
    
    print(v.name, "=", v.varValue)
    skład=[":".join((v.nazwa, str(v.varValue))) for v.nazwa in ["wieprzowinę","kurczaka","sardynki"]  ]
print("Total Cost of Ingredients per can = ", value(prob.objective))



print("Końcowy produkt będzie kosztować", (value(prob.objective)), "będzie miał w składzie",skład)




Status: Optimal
Total Cost of Ingredients per can =  1.0912276
Końcowy produkt będzie kosztować 1.0912276 będzie miał w składzie ['wieprzowinę:2.63158', 'kurczaka:2.63158', 'sardynki:2.63158']


In [16]:
from pulp import *

In [96]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL','SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES':0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES':0.750}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
               'SARDINES': 0.001}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES':0.002}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES':0.05}
# A dictionary of the vitamin B in each of the Ingredients is created
vitaminBPercent = {'CHICKEN': 0.3, 
               'BEEF': 0.26, 
               'MUTTON': 0.26, 
               'RICE': 0.00, 
               'WHEAT': 0.00, 
               'GEL': 0.000,
               'SARDINES':0.89}


# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "VitaminBRequirement"
prob.writeLP("WhiskasModel.lp")
prob.solve()

print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print("Total Cost of Ingredients per can = ", value(prob.objective))

Status: Optimal
Ingr_BEEF = 59.98
Ingr_CHICKEN = 0.0
Ingr_GEL = 38.018
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_SARDINES = 2.002
Ingr_WHEAT = 0.0
Total Cost of Ingredients per can =  0.51986


In [3]:
from pulp import *



In [5]:

prob = LpProblem("The Windowlem",LpMaximize)
x1=LpVariable("f1",0,None)
x2=LpVariable("f2",0)
zysk=LpVariable("zysk",0)
prob += 1*x1 <=4
prob += x2*2<=12
prob += 3*x1+2*x2<=18
prob += *3000 + (x2/4)*5000==zysk

TypeError: unsupported operand type(s) for /: 'LpVariable' and 'int'